# Querying XML In SQL Server - Cheat Sheet

This notebook provides examples of common patterns used when querying XML data in SQL Server. The samples should work on any version of SQL Server from 2005 onwards and require no setup other than permissions to query in tempdb.

It was created by [Barney Lawrence](https://barneylawrence.com/) the latest version can be found in the GitHub repository [BarneyLawrence/XML-Queries-In-SQL-Server](https://github.com/BarneyLawrence/XML-Queries-In-SQL-Server)

For additional detail visit my blog series on the subject.

- [Part 1 – Introduction](https://barneylawrence.com/2021/03/15/querying-xml-in-sql-server-part-1-introduction/)
- [Part 2 – Simple Paths and Attributes With the Value Method](https://barneylawrence.com/2021/03/23/querying-xml-in-sql-server-part-2-simple-paths-and-attributes-with-the-value-method/)
- [Part 3 – Handling Repeating Regions With the Nodes Method](https://barneylawrence.com/2021/03/30/querying-xml-in-sql-server-part-3-handling-repeating-regions-with-the-nodes-method/)
- [Part 4 – Filtering With The Exist Method](https://barneylawrence.com/2021/04/07/querying-xml-in-sql-server-part-4-filtering-with-the-exist-method/)
- [Part 5 – Introduction to the Query Method and XQuery](https://barneylawrence.com/2021/04/21/querying-xml-in-sql-server-part-5-an-introduction-to-the-query-method-and-xquery/)
- [Part 6 – Sequencing Elements and Finding Positions](https://barneylawrence.com/2021/04/28/querying-xml-in-sql-server-part-6-sequencing-elements-and-finding-positions/)
- [Part 7 – Creating XML](https://barneylawrence.com/2021/05/06/querying-xml-in-sql-server-part-7-creating-xml/)

## Getting Started

The first code section stores sample data in a temporary table, run this first as subsequent sections depend on it.

In [1]:
CREATE TABLE #MyXML (MyXML XML NOT NULL);
 
INSERT INTO #MyXML(MyXML)
VALUES
(
'
<Order Customer="Barney">
    <Pizza Size="Medium">
        <Topping>Ham</Topping>
        <Topping>Pineapple</Topping>
    </Pizza>
    <Pizza Size="Large">
        <Topping>Jalapenos</Topping>
        <Topping>Onions</Topping>
        <Topping>Mushrooms</Topping>
    </Pizza>
    <IceCream Size="Large">
        <Topping>Pineapple</Topping>
        <Topping>Chocolate</Topping>
    </IceCream>
</Order>
'
);

SELECT MyXML
FROM #MyXML;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.085

MyXML
"<Order Customer=""Barney""><Pizza Size=""Medium""><Topping>Ham</Topping><Topping>Pineapple</Topping></Pizza><Pizza Size=""Large""><Topping>Jalapenos</Topping><Topping>Onions</Topping><Topping>Mushrooms</Topping></Pizza><IceCream Size=""Large""><Topping>Pineapple</Topping><Topping>Chocolate</Topping></IceCream></Order>"


# Simple Value Extraction

These examples show methods of specifiying a specific path through nested XML tags to reach a value.

In [2]:
SELECT
--Get the value inside an element
X.MyXML.value('(Order/Pizza/Topping)[1]','varchar(500)') AS FirstPizzaTopping,
--Get an attribute on a tag
X.MyXML.value('(Order/Pizza/@Size)[1]','varchar(500)') AS FirstPizzaSize,
--Be specific about a path
X.MyXML.value('(Order/Pizza[2]/Topping[2])[1]','varchar(500)') AS SecondPizzaSecondTopping,
--Get the last element
X.MyXML.value('(Order/Pizza/Topping)[last()]','varchar(500)') AS LastPizzaTopping,
--Find elements regardless of parents 
X.MyXML.value('(//Topping)[last()]','varchar(500)') AS LastTopping,
--Use a wildcard in the path
X.MyXML.value('(Order/*/Topping)[last()]','varchar(500)') AS LastTopping2
FROM #MyXML AS X;

(1 row affected)

Total execution time: 00:00:00.046

FirstPizzaTopping,FirstPizzaSize,SecondPizzaSecondTopping,LastPizzaTopping,LastTopping,LastTopping2
Ham,Medium,Onions,Mushrooms,Chocolate,Chocolate


# Searching Values

These examples show how to identify values in an XML document based on the values of other attributes in the document.

In [3]:
DECLARE @MyTopping varchar(50) = 'Ham'
SELECT
    --Choose an item based on values on the parent
    X.MyXML.value('(Order/Pizza[@Size="Large"]/Topping)[1]','varchar(500)') AS FirstLargeTopping,
    --Choose an item based on values on the child
    X.MyXML.value('(Order/Pizza[./Topping = "Onions"]/@Size)[1]','varchar(500)') AS FirstPizzaWithOnions,
    --As above with a varible for the topping
    X.MyXML.value('(Order/*[./Topping = sql:variable("@MyTopping")]/@Size)[1]','varchar(500)') AS FirstThingWithVariableTopping,
    --Find the name of the first tag with a chocolate topping
    X.MyXML.value('local-name((Order/*[./Topping = "Chocolate"])[1])','varchar(500)') AS FirstThingWithChocolate
FROM #MyXML AS X;

(1 row affected)

Total execution time: 00:00:00.062

FirstLargeTopping,FirstPizzaWithOnions,FirstThingWithVariableTopping,FirstThingWithChocolate
Jalapenos,Large,Medium,IceCream


# Splitting XML Into Rows

These next examples show how to break a single XML document into multiple rows.

Examples use CROSS APPLY but OUTER APPLY can also be used, particularly where sub elements may be optional.

## Calling The Nodes Method Multiple Times

In this approach we split into rows at the first sub level and then apply the nodes method a second time to split at the second level.

Values at both levels can be accessed directly from their respective results.

In [4]:
SELECT
    Pizzas.PizzaXML.value('(@Size)','varchar(500)') AS PizzaSize,
    Toppings.ToppingXML.value('(.)','varchar(500)') AS Topping
FROM #MyXML AS X
CROSS APPLY X.MyXML.nodes('Order/Pizza') AS Pizzas(PizzaXML)
OUTER APPLY Pizzas.PizzaXML.nodes('Topping') AS Toppings(ToppingXML);

(5 rows affected)

Total execution time: 00:00:00.022

PizzaSize,Topping
Medium,Ham
Medium,Pineapple
Large,Jalapenos
Large,Onions
Large,Mushrooms


## Calling The Nodes Method Directly At The Bottom Level

This version goes directly to the bottom level and then moves up using '..' to obtain values at higher levels. Because we don't specify the parent of the toppings we get both pizza and ice cream

The local-name function allows us to obtain the name of the element for each row.

In [5]:
SELECT
    Toppings.ToppingXML.value('local-name(..)','varchar(500)') AS ItemType,
    Toppings.ToppingXML.value('(../@Size)','varchar(500)') AS Size,
    Toppings.ToppingXML.value('(.)','varchar(500)') AS Topping
FROM #MyXML AS X
CROSS APPLY X.MyXML.nodes('//Topping') AS Toppings(ToppingXML);

(7 rows affected)

Total execution time: 00:00:00.026

ItemType,Size,Topping
Pizza,Medium,Ham
Pizza,Medium,Pineapple
Pizza,Large,Jalapenos
Pizza,Large,Onions
Pizza,Large,Mushrooms
IceCream,Large,Pineapple
IceCream,Large,Chocolate


# Using the Exist Method

Exist checks to see if a query returns a result or a null and then returns a 1 or a 0.

Here it is used to peek down to the topping level to check the types of topping used without having to use the nodes method to change the grain of the query.

In [6]:
SELECT
    Items.ItemXML.value('local-name(.)','varchar(500)') AS ItemType,
    Items.ItemXML.value('(@Size)','varchar(500)') AS ItemSize,
    Items.ItemXML.exist('.[./Topping = "Pineapple"]') AS IsFruity,
    Items.ItemXML.exist('.[./Topping = "Jalapenos"]') AS IsSpicy
FROM #MyXML AS X
CROSS APPLY X.MyXML.nodes('Order/*') AS Items(ItemXML)

(3 rows affected)

Total execution time: 00:00:00.037

ItemType,ItemSize,IsFruity,IsSpicy
Pizza,Medium,1,0
Pizza,Large,0,1
IceCream,Large,1,0


# Using The Query Method And XQuery

Query returns XML as a result. Some of these examples could also be used in the values method to return text or a number.

The sequence number example works by assigning the current element to a variable and then counting the number of elements that preceed it within its parent element (effectively counting elder siblings). It will be zero based as the first element has none preceeding it.

In [8]:
SELECT
	ItemXML.value('(@Size)','varchar(500)') AS ItemSize,
	--Show the current element produced by the nodes method
	ItemXML.query('.') AS ToppingXML,
	--Count the number of toppings
	ItemXML.query('count(./Topping)') AS ToppingCount,
	--Produce a list of toppings
	ItemXML.query(
		'for $T in ./Topping return string($T)'
		) AS ToppingList,
	--Generate a sequence number
	ItemXML.query(
	'let $P := . return count(../*[. << $P])'
		) AS ItemSequence
FROM #MyXML AS X
CROSS APPLY 
	X.MyXML.nodes('Order/*') AS Item(ItemXML);


(3 rows affected)

Total execution time: 00:00:00.016

ItemSize,ToppingXML,ToppingCount,ToppingList,ItemSequence
Medium,"<Pizza Size=""Medium""><Topping>Ham</Topping><Topping>Pineapple</Topping></Pizza>",2,Ham Pineapple,0
Large,"<Pizza Size=""Large""><Topping>Jalapenos</Topping><Topping>Onions</Topping><Topping>Mushrooms</Topping></Pizza>",3,Jalapenos Onions Mushrooms,1
Large,"<IceCream Size=""Large""><Topping>Pineapple</Topping><Topping>Chocolate</Topping></IceCream>",2,Pineapple Chocolate,2
